# Best Methods to Deal with Common Data Problems
## Mastering the most time-consuming skill, part I
<img src='images/clean2.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@thecreative_exchange?utm_source=medium&utm_medium=referral'>Steve Johnson</a>
        on 
        <a href='https://unsplash.com/?utm_source=medium&utm_medium=referral'>Unsplash</a>
    </strong>
</figcaption>